## **Esempi: Multi-AI Agents per prenotare hotel**

Nel mondo frenetico di oggi, organizzare un viaggio di lavoro non significa solo prenotare un volo e una camera d'albergo. Richiede un livello di coordinamento ed efficienza che può essere difficile da raggiungere. È qui che entrano in gioco i Multi-AI Agents, rivoluzionando il modo in cui gestiamo le nostre esigenze di viaggio.

Immagina di avere a disposizione un team di agenti intelligenti, che lavorano insieme per gestire ogni aspetto del tuo viaggio con precisione e facilità. Con la nostra tecnologia avanzata di intelligenza artificiale, abbiamo creato agenti specializzati per i servizi di prenotazione e l'organizzazione dell'itinerario, garantendo un'esperienza di viaggio fluida e senza stress.

Questo è uno scenario di base. Quando si pianifica un viaggio di lavoro, è necessario consultare un agente di viaggi aziendali per ottenere informazioni sui biglietti aerei, sugli hotel, ecc. Attraverso gli AI Agents, possiamo creare agenti per i servizi di prenotazione e agenti per l'organizzazione dell'itinerario che collaborano tra loro per migliorare il livello di intelligenza.


# Inizializzare il Servizio Azure AI Agent e ottenere informazioni di configurazione da **.env**

### **.env** 

Crea un file .env 

**.env** contiene la stringa di connessione del Servizio Azure AI Agent, il modello utilizzato da AOAI e il corrispondente servizio API di ricerca di Google, ENDPOINT, ecc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nome del Deployment del Modello del Servizio Azure AI Agent"

[**NOTE**] Avrai bisogno di un modello con un limite di 100.000 Token al minuto e un limite di 600 richieste al minuto.

  Puoi ottenere il modello in Azure AI Foundry - Modello ed Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Stringa di Connessione del Progetto del Servizio Azure AI Agent"

  Puoi ottenere la stringa di connessione del progetto nella panoramica del progetto nella schermata del portale AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "La tua chiave API di ricerca SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Il tuo endpoint di ricerca SERPAPI"

Per ottenere il Nome del Deployment del Modello e la Stringa di Connessione del Progetto del Servizio Azure AI Agent, è necessario creare il Servizio Azure AI Agent. Si consiglia di utilizzare [questo template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) per crearlo direttamente （***Nota:*** Il Servizio Azure AI Agent è attualmente disponibile in regioni limitate. Si consiglia di fare riferimento a [questo link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) per impostare la regione)

L'Agent deve accedere a SERPAPI. Si consiglia di registrarsi utilizzando [questo link](https://serpapi.com/searches). Dopo la registrazione, puoi ottenere una chiave API e un endpoint unici.


# Accedi ad Azure

Ora devi accedere ad Azure. Apri un terminale in VScode ed esegui il comando `az login`.


# Configurazione

Per eseguire questo notebook, sarà necessario installare le seguenti librerie. Ecco un elenco delle librerie richieste e i relativi comandi pip install:

azure-identity: Per l'autenticazione su Azure.  
requests: Per effettuare richieste HTTP.  
semantic-kernel: Per il framework del kernel semantico (supponendo che sia una libreria personalizzata o specifica, potrebbe essere necessario installarla da una fonte o repository specifico).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Spiegazione: 
import asyncio: Questo importa il modulo asyncio, che fornisce supporto per la programmazione asincrona in Python. Ti consente di scrivere codice concorrente utilizzando la sintassi async e await.  
from typing: Annotated: Questo importa il tipo Annotated dal modulo typing. Annotated viene utilizzato per aggiungere metadati agli hint di tipo, che possono essere utili per vari scopi come la validazione, la documentazione o gli strumenti.


In [ ]:
import asyncio,os
from typing import Annotated

# Spiegazione:
Utilizzando from dotenv import load_dotenv e load_dotenv(), puoi gestire facilmente le impostazioni di configurazione e le informazioni sensibili (come chiavi API e URL di database) in un file .env, mantenendole separate dal tuo codice sorgente e rendendo la tua applicazione più sicura e facile da configurare.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Spiegazione:

Importazione: from azure.identity.aio import DefaultAzureCredential: Questo importa la classe DefaultAzureCredential dal modulo azure.identity.aio. La parte "aio" nel nome del modulo indica che è progettato per operazioni asincrone.

Scopo di DefaultAzureCredential: La classe DefaultAzureCredential fa parte dell'SDK di Azure per Python. Fornisce un metodo predefinito per autenticarsi con i servizi Azure. Tenta di autenticarsi utilizzando diversi metodi in un ordine specifico, come variabili d'ambiente, identità gestite e credenziali di Azure CLI.

Operazioni Asincrone: Il modulo "aio" indica che la classe DefaultAzureCredential supporta operazioni asincrone. Questo significa che puoi usarla con asyncio per eseguire richieste di autenticazione non bloccanti.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Spiegazione:
Importa vari moduli e classi dal pacchetto semantic_kernel. Ecco una panoramica di ciascun import:

AgentGroupChat da semantic_kernel.agents: Questa classe gestisce le funzionalità relative alla chat di gruppo per agenti AI. AzureAIAgent e AzureAIAgentSettings da semantic_kernel.agents.azure_ai

AzureAIAgent: Questa classe viene utilizzata per creare e gestire agenti AI che utilizzano i servizi Azure AI.

AzureAIAgentSettings: Questa classe viene utilizzata per configurare le impostazioni dell'AzureAIAgent. TerminationStrategy da semantic_kernel.agents.strategies.termination.termination_strategy:

Questa classe definisce strategie per terminare l'esecuzione degli agenti AI in determinate condizioni. ChatMessageContent da semantic_kernel.contents.chat_message_content:

Questa classe viene utilizzata per gestire il contenuto dei messaggi in chat.
AuthorRole da semantic_kernel.contents.utils.author_role:

Questa classe definisce diversi ruoli per gli autori nel contesto dei messaggi in chat.

kernel_function da semantic_kernel.functions.kernel_function_decorator: Questo decoratore viene utilizzato per definire funzioni kernel, ovvero funzioni che possono essere eseguite all'interno del framework semantic kernel.
Questi import configurano i componenti necessari per creare e gestire agenti AI che possono interagire in un ambiente di chat di gruppo, possibilmente per attività come la prenotazione di hotel o attività simili.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Spiegazione:
Successivamente importiamo la classe CodeInterpreterTool dal modulo azure.ai.projects.models.

CodeInterpreterTool: Questa classe fa parte dell'SDK di Azure AI ed è utilizzata per interpretare ed eseguire codice nel contesto dei progetti di intelligenza artificiale. Fornisce funzionalità per eseguire frammenti di codice, analizzare il codice o integrare l'esecuzione del codice nei flussi di lavoro di AI.  
Questo import configura il componente necessario per utilizzare CodeInterpreterTool nel tuo progetto, che potrebbe essere utile per attività che coinvolgono l'interpretazione e l'esecuzione dinamica del codice.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Spiegazione:  
La classe ApprovalTerminationStrategy fornisce una strategia specifica per terminare l'operazione di un agente AI. L'agente si fermerà se l'ultimo messaggio nella sua cronologia di interazione contiene la parola "salvato". Questo potrebbe essere utile in scenari in cui il compito dell'agente è considerato completato una volta ricevuta la conferma che qualcosa è stato "salvato". Definire il metodo di interazione. Dopo che il piano di prenotazione è stato salvato, può essere interrotto al ricevimento del segnale "salvato".


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Spiegazione:

La riga di codice inizializza un oggetto AzureAIAgentSettings con impostazioni predefinite o standard, chiamando il metodo create(). Questo oggetto di impostazioni (ai_agent_settings) può essere utilizzato per configurare e gestire un'istanza di AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Spiegazione:
Importando la libreria requests, puoi facilmente effettuare richieste HTTP e interagire con i servizi web nel tuo codice Python.


In [ ]:
import requests

# Spiegazione:
Questa è una variabile che memorizza la chiave API per accedere a un servizio API SERP (Search Engine Results Page). Una chiave API è un identificatore unico utilizzato per autenticare le richieste associate al tuo account.

'GOOGLE_SEARCH_API_KEY': Questa è una stringa segnaposto. Devi sostituire 'GOOGLE_SEARCH_API_KEY' con la tua chiave API SERP effettiva.

Scopo: Lo scopo di questa riga è memorizzare la chiave API in una variabile, in modo che possa essere utilizzata per autenticare le richieste al servizio API SERP. La chiave API è necessaria per accedere al servizio e effettuare ricerche.

Come ottenere una chiave API SERP: Per ottenere una chiave API SERP, segui questi passaggi generali su https://serpapi.com (i passaggi esatti possono variare a seconda del servizio API SERP specifico che stai utilizzando):

Scegli un servizio API SERP: Esistono diversi servizi API SERP disponibili, come SerpAPI, Google Custom Search JSON API e altri. Scegli quello che meglio soddisfa le tue esigenze.

Registrati per un account:

Vai al sito web del servizio API SERP scelto https://www.serpapi.com e registrati per un account. Potrebbe essere necessario fornire alcune informazioni di base e verificare il tuo indirizzo email.

Crea una chiave API:

Dopo esserti registrato, accedi al tuo account e vai alla sezione API o al dashboard. Cerca un'opzione per creare o generare una nuova chiave API.
Copia la chiave API:

Una volta generata la chiave API, copiala. Questa chiave sarà utilizzata per autenticare le tue richieste al servizio API SERP.
Sostituisci il segnaposto:

Sostituisci il segnaposto nel tuo file .env


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Spiegazione:
La classe BookingPlugin fornisce metodi per prenotare hotel e voli utilizzando l'API di ricerca Google di Serpapi.com. Costruisce i parametri necessari, invia richieste API e processa le risposte per restituire informazioni pertinenti sulle prenotazioni. La chiave API (SERPAPI_SEARCH_API_KEY) e l'endpoint (SERPAPI_SEARCH_ENDPOINT) vengono utilizzati per autenticare e inviare richieste all'API di ricerca Google.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Spiegazione:
La classe SavePlugin fornisce un metodo saving_plan per salvare i piani di viaggio utilizzando i servizi Azure AI. Configura le credenziali Azure, crea un agente AI, elabora gli input dell'utente per generare e salvare il contenuto del piano di viaggio, e gestisce le operazioni di salvataggio dei file e di pulizia. Il metodo restituisce "Salvato" al completamento con successo.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Spiegazione:
Questo codice configura agenti Azure AI per gestire la prenotazione di voli e hotel, e salvare piani di viaggio basati sugli input degli utenti. Utilizza credenziali Azure per creare e configurare gli agenti, elabora gli input degli utenti tramite una chat di gruppo, e garantisce una corretta pulizia dopo il completamento dei compiti. Gli agenti utilizzano plugin specifici (BookingPlugin e SavePlugin) per svolgere i rispettivi compiti.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche potrebbero contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale eseguita da un traduttore umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
